In [1]:
import dask.array as da
from dask import delayed, compute
from dask.distributed import Client
import numpy as np
import nd2

In [2]:
# Initialize dask
client = Client(n_workers=4)

In [ ]:
f = nd2.ND2File("../data/A1_s3001.nd2")
array = f.to_dask()
array

In [ ]:
data = array[0:200:,:,:]
data

In [ ]:
%%time

results = []
for image in data:    
    x = np.fft.fft2(image)
    results.append(x)

total = np.cumsum(np.abs(results)**2)
result_1a = total

In [ ]:
%%time

x = np.fft.fft2(data)
total = np.cumsum(np.abs(x)**2)
result_1b = total

In [ ]:
%%time

results = []
for image in data:
    x = delayed(np.fft.fft2(image))
    y = np.abs(x)**2
    results.append(y)

total = delayed(np.cumsum)(results)
result_2a = total.compute()

In [ ]:
%%time

x = delayed(np.fft.fft2(data))
y = np.abs(x)**2

total = delayed(np.cumsum)(y)
result_2b = total.compute()

In [7]:
f = nd2.ND2File("../data/A1_s3001.nd2")
xarr = f.to_xarray(delayed=True)
xarr

<xarray.DataArray '_dask_block-18a85d63436af8c4dcdc3f01140ba365' (T: 5000,
                                                                  Y: 512, X: 512)>
dask.array<_dask_block, shape=(5000, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * X        (X) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * T        (T) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
Attributes:
    metadata:  {'metadata': Metadata(contents=Contents(channelCount=1, frameC...

In [4]:
import tensorflow as tf
import torch

In [14]:
%%time
for image in xarr[0:100,:,:]:
    np_arr = image.to_numpy().astype('int64')
    tensor = torch.from_numpy(np_arr)
    x = torch.fft.fft2(tensor)
    y = np.abs(x)**2


total = np.cumsum(y)
result_3 = total


CPU times: total: 33.7 s
Wall time: 18.7 s


In [ ]:
# np.testing.assert_equal(result_1a, result_1b)
# np.testing.assert_equal(result_2a, result_2b)
# np.testing.assert_equal(result_1a, result_2a)

In [15]:
result_3

tensor([6.0349e+19, 6.0351e+19, 6.0351e+19,  ..., 6.0371e+19, 6.0371e+19,
        6.0371e+19])